In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re

def extract_all_text_as_array(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text = soup.stripped_strings
        all_text_array = list(all_text)
        return all_text_array
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve page. Error: {e}")
        return None

def check_dark_pattern(url, dark_pattern_strings):
    extracted_text = extract_all_text_as_array(url)
    if extracted_text:
        for pattern in dark_pattern_strings:
            if any(pattern.lower() in text.lower() for text in extracted_text):
                return True, ' '.join(extracted_text)
    return False, ' '.join(extracted_text[:10]) if extracted_text else ''

def main():
    # Load dark pattern strings from the dataset
    dark_pattern_strings = set()
    with open('dark-patterns.csv', 'r') as dark_pattern_file:
        dark_pattern_reader = csv.DictReader(dark_pattern_file)
        for row in dark_pattern_reader:
            if row['Pattern String']:  # Ensure it's not an empty row
                dark_pattern_strings.add(row['Pattern String'])

    # Load ecommerce links from CSV
    ecommerce_links = []
    with open('ecommerce_links.csv', 'r') as ecommerce_links_file:
        ecommerce_links_reader = csv.DictReader(ecommerce_links_file)
        for row in ecommerce_links_reader:
            if row['Link']:  # Ensure it's not an empty row
                has_dark_pattern, pattern_string = check_dark_pattern(row['Link'], dark_pattern_strings)
                row['Deceptive?'] = '1' if has_dark_pattern else '0'
                row['Pattern String'] = pattern_string

                # Print the results
                print(f"Link: {row['Link']}, Deceptive?: {row['Deceptive?']}, Pattern String: {row['Pattern String']}")

                ecommerce_links.append(row)

    # Write the updated results back to a new CSV file
    with open('ecommerce_links_updated.csv', 'w', newline='') as updated_file:
        fieldnames = ['Link', 'Deceptive?', 'Pattern String']
        writer = csv.DictWriter(updated_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(ecommerce_links)

if __name__ == "__main__":
    main()


Link: http://amazon.in/, Deceptive?: 1, Pattern String: Online Shopping site in India: Shop Online for Mobiles, Books, Watches, Shoes and More - Amazon.in Skip to main content .in Delivering to Mumbai 400001 Update location All Select the department you want to search in All Categories Alexa Skills Amazon Devices Amazon Fashion Amazon Fresh Amazon Pharmacy Appliances Apps & Games Audible Audiobooks Baby Beauty Books Car & Motorbike Clothing & Accessories Collectibles Computers & Accessories Electronics Furniture Garden & Outdoors Gift Cards Grocery & Gourmet Foods Health & Personal Care Home & Kitchen Industrial & Scientific Jewellery Kindle Store Luggage & Bags Luxury Beauty Movies & TV Shows Music Musical Instruments Office Products Pet Supplies Prime Video Shoes & Handbags Software Sports, Fitness & Outdoors Subscribe & Save Tools & Home Improvement Toys & Games Under ₹500 Video Games Watches Search Amazon.in EN Hello, sign in Account & Lists Returns & Orders Cart All Fresh Amazon m

In [3]:
# prompt: drop all the rows of the ecommerce_links_updated.csv where column "deceptive?"is 1

import csv

# Open the input CSV file for reading
with open('./sample_data/ecommerce_links_updated.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('ecommerce_links_updated_without_deceptive.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Deceptive?" column is not "1"
            if row['Deceptive?'] != '1':
                # Write the row to the output CSV file
                csv_writer.writerow(row)


In [6]:
# prompt: drop all the rows of the ecommerce_links_updated.csv where column  pattern string is empty

import csv

# Open the input CSV file for reading
with open('./sample_data/ecommerce_links_updated_without_deceptive.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('ecommerce_links_updated_without_empty_pattern_string.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Pattern String" column is not empty
            if row['Pattern String']:
                # Write the row to the output CSV file
                csv_writer.writerow(row)


In [21]:
import csv

# Open the input CSV files
with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file, open('./sample_data/ecommerce_links_updated_without_empty_pattern_string.csv', 'r') as ecommerce_file:
    dark_pattern_reader = csv.DictReader(dark_pattern_file)
    ecommerce_reader = csv.DictReader(ecommerce_file)

    # Create a list to store combined rows
    combined_rows = []

    # Read and mark the first 100 rows from dark patterns
    for i, dark_pattern_row in enumerate(dark_pattern_reader):
        if i >= 100:
            break
        row = {
            'Link': dark_pattern_row['Website Page'],
            'Pattern String': dark_pattern_row['Pattern String'],
            'Deceptive?': '1'  # Mark as deceptive
        }
        combined_rows.append(row)

    # Read rows from ecommerce and add to the list
    for ecommerce_row in ecommerce_reader:
        combined_rows.append(ecommerce_row)

    # Open the output CSV file
    with open('combined_datsa.csv', 'w', newline='') as combined_file:
        fieldnames = ['Link', 'Pattern String', 'Deceptive?']
        combined_writer = csv.DictWriter(combined_file, fieldnames=fieldnames)
        combined_writer.writeheader()

        # Write the combined rows to the output file
        for row in combined_rows:
            combined_writer.writerow(row)


In [24]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_all_text_as_array(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text = soup.stripped_strings
        all_text_array = list(all_text)
        return all_text_array
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve page. Error: {e}")
        return None

def check_dark_pattern(url, dark_pattern_strings):
    extracted_text = extract_all_text_as_array(url)
    if extracted_text:
        for pattern in dark_pattern_strings:
            if any(pattern.lower() in text.lower() for text in extracted_text):
                return True, ' '.join(extracted_text)
    return False, ' '.join(extracted_text[:10]) if extracted_text else ''

def main():
    # Load dark pattern strings from the dataset
    dark_pattern_strings = set()
    with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file:
        dark_pattern_reader = csv.DictReader(dark_pattern_file)
        for row in dark_pattern_reader:
            if row['Pattern String']:  # Ensure it's not an empty row
                dark_pattern_strings.add(row['Pattern String'])

    # Load links from combined CSV
    combined_links = []
    with open('./sample_data/combined_links.csv', 'r') as combined_links_file:
        combined_links_reader = csv.DictReader(combined_links_file)
        for row in combined_links_reader:
            if row['Link']:  # Ensure it's not an empty row
                has_dark_pattern, pattern_string = check_dark_pattern(row['Link'], dark_pattern_strings)
                row['Deceptive?'] = '1' if has_dark_pattern else '0'
                row['Pattern String'] = pattern_string

                # Print the results
                print(f"Link: {row['Link']}, Deceptive?: {row['Deceptive?']}, Pattern String: {row['Pattern String']}")

                combined_links.append(row)

    # Write the updated results back to a new CSV file
    with open('combined_links_updated.csv', 'w', newline='') as updated_file:
        fieldnames = ['Link', 'Deceptive?', 'Pattern String']
        writer = csv.DictWriter(updated_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(combined_links)

if __name__ == "__main__":
    main()


Link: https://www.cettire.com/, Deceptive?: 0, Pattern String: 
Link: https://www.fashionnova.com/, Deceptive?: 0, Pattern String: Fashion Nova | Fashion Online For Women | Affordable Women's Clothing | Fashion Nova Skip to content women plus+curve men kids beauty Search Search Clear
Link: https://www.gymshark.com/, Deceptive?: 0, Pattern String: Gymshark Official Store - Shop Gym Clothes & Workout Clothes Accessibility Statement Skip to content more account Accessibility Statement Help Email Sign Up Blog Store Selector dropdown
Link: https://shop.ledger.com/, Deceptive?: 1, Pattern String: Ledger - Home of the first and only certified Hardware wallets | Ledger New: Enjoy wallet recovery made easy with Ledger Recover, provided by Coincover Get started Products Ledger Stax Ledger Nano X Ledger Nano S Plus Compare our devices Packs Accessories Collaborations See All Products Download Ledger Live Supported crypto App and services Ledger Live Ledger Recover The CL Card Supported Services C

In [26]:
#  drop all the rows in combined_links_updated.csv where deceptive? column equal to 1

import csv

# Open the input CSV file for reading
with open('./combined_links_updated.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('combined_links_without_deceptive.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Deceptive?" column is not "1"
            if row['Deceptive?'] != '1':
                # Write the row to the output CSV file
                csv_writer.writerow(row)

In [32]:
import csv

# Open the input CSV files
with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file, open('combined_links_without_deceptive.csv', 'r') as combined_links_file, open('./sample_data/ecommerce_links_updated_without_empty_pattern_string.csv', 'r') as ecommerce_links_file:
    dark_pattern_reader = csv.DictReader(dark_pattern_file)
    combined_links_reader = csv.DictReader(combined_links_file)
    ecommerce_links_reader = csv.DictReader(ecommerce_links_file)

    # Create a list to store combined rows
    combined_rows = []

    # Read all rows from combined links without deceptive
    for combined_row in combined_links_reader:
        combined_rows.append(combined_row)

    # Read the last 100 rows from dark patterns
    dark_pattern_rows = list(dark_pattern_reader)[-100:]

    # Append the last 100 rows from dark patterns to the combined rows list
    for dark_pattern_row in dark_pattern_rows:
        row = {
            'Link': dark_pattern_row['Website Page'],
            'Pattern String': dark_pattern_row['Pattern String'],
            'Deceptive?': '1'  # Mark as deceptive
        }
        combined_rows.append(row)

    # Read all rows from ecommerce links without empty pattern string
    for ecommerce_row in ecommerce_links_reader:
        combined_rows.append(ecommerce_row)

    # Open the output CSV file
    with open('./combined_datsa.csv', 'w', newline='') as combined_file:
        fieldnames = ['Link', 'Pattern String', 'Deceptive?']
        combined_writer = csv.DictWriter(combined_file, fieldnames=fieldnames)
        combined_writer.writeheader()

        # Write the combined rows to the output file
        combined_writer.writerows(combined_rows)


In [35]:
import csv

# Open the input CSV files
with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file, open('./combined_datsa.csv', 'a', newline='') as combined_file:
    dark_pattern_reader = csv.DictReader(dark_pattern_file)
    combined_writer = csv.DictWriter(combined_file, fieldnames=['Link', 'Pattern String', 'Deceptive?'])

    # Read all rows from dark patterns
    dark_pattern_rows = list(dark_pattern_reader)[-70:]

    # Append the last 70 rows from dark patterns to the combined data file
    for dark_pattern_row in dark_pattern_rows:
        row = {
            'Link': dark_pattern_row['Website Page'],
            'Pattern String': dark_pattern_row['Pattern String'],
            'Deceptive?': '1'  # Mark as deceptive
        }
        combined_writer.writerow(row)


In [36]:
# prompt: drop the rows in combined_datsa.csv where pattern string coulumn is empty

import pandas as pd

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('combined_datsa.csv')

# Drop rows where the 'Pattern String' column is empty
df = df.dropna(subset=['Pattern String'])

# Save the updated DataFrame to a new CSV file
df.to_csv('combined_datsa_without_empty_pattern_string.csv', index=False)


In [38]:
import requests
from bs4 import BeautifulSoup
import csv

def get_links_from_ecommerce_list(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a', {'data-testid': 'custom-anchor-tag'})
        ecommerce_links = [link.get('href') for link in links]
        return ecommerce_links
    else:
        print(f"Failed to retrieve page. Status code: {response.status_code}")
        return []

def save_links_to_csv(links, csv_filename='ecommerce_links_more.csv'):
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = ['Link']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for link in links:
            writer.writerow({'Link': link})

if __name__ == "__main__":
    ecommerce_links = get_links_from_ecommerce_list('https://www.bigcommerce.com/showcase/')
    save_links_to_csv(ecommerce_links)

    print("Links saved to ecommerce_links_more.csv")


Links saved to ecommerce_links_more.csv


In [39]:

import csv

# Open the input CSV file for reading
with open('ecommerce_links_more.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('without_bigcommerce.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Link" column starts with "https://www.bigcommerce"
            if not row['Link'].startswith('https://www.bigcommerce'):
                # Write the row to the output CSV file
                csv_writer.writerow(row)


In [41]:
import requests
from bs4 import BeautifulSoup
import csv

def extract_all_text_as_array(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        all_text = soup.stripped_strings
        all_text_array = list(all_text)
        return all_text_array
    except requests.exceptions.RequestException as e:
        print(f"Failed to retrieve page. Error: {e}")
        return None

def check_dark_pattern(url, dark_pattern_strings):
    extracted_text = extract_all_text_as_array(url)
    if extracted_text:
        for pattern in dark_pattern_strings:
            if any(pattern.lower() in text.lower() for text in extracted_text):
                return True, ' '.join(extracted_text)
    return False, ' '.join(extracted_text[:10]) if extracted_text else ''

def main():
    # Load dark pattern strings from the dataset
    dark_pattern_strings = set()
    with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file:
        dark_pattern_reader = csv.DictReader(dark_pattern_file)
        for row in dark_pattern_reader:
            if row['Pattern String']:  # Ensure it's not an empty row
                dark_pattern_strings.add(row['Pattern String'])

    # Load links from "without_bigcommerce.csv"
    without_bigcommerce_links = []
    with open('./without_bigcommerce.csv', 'r') as without_bigcommerce_file:
        without_bigcommerce_reader = csv.DictReader(without_bigcommerce_file)
        for row in without_bigcommerce_reader:
            if row['Link']:  # Ensure it's not an empty row
                has_dark_pattern, pattern_string = check_dark_pattern(row['Link'], dark_pattern_strings)
                row['Deceptive?'] = '1' if has_dark_pattern else '0'
                row['Pattern String'] = pattern_string

                # Print the results
                print(f"Link: {row['Link']}, Deceptive?: {row['Deceptive?']}, Pattern String: {row['Pattern String']}")

                without_bigcommerce_links.append(row)

    # Write the updated results back to a new CSV file
    with open('without_bigcommerce_updated.csv', 'w', newline='') as updated_file:
        fieldnames = ['Link', 'Deceptive?', 'Pattern String']
        writer = csv.DictWriter(updated_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(without_bigcommerce_links)

if __name__ == "__main__":
    main()


Failed to retrieve page. Error: Invalid URL '/request-a-demo/': No scheme supplied. Perhaps you meant https:///request-a-demo/?
Link: /request-a-demo/, Deceptive?: 0, Pattern String: 
Failed to retrieve page. Error: Invalid URL '/request-a-demo/': No scheme supplied. Perhaps you meant https:///request-a-demo/?
Link: /request-a-demo/, Deceptive?: 0, Pattern String: 
Link: https://www.tedbaker.com/uk/, Deceptive?: 0, Pattern String: Homepage | Ted Baker UK You need to enable Javascript to use this site. Search Ted Suggests Sale Womens Further Reductions Women's Clothing Dresses Coats & Jackets Tops
Link: https://www.tedbaker.com/uk/, Deceptive?: 0, Pattern String: Homepage | Ted Baker UK You need to enable Javascript to use this site. Search Ted Suggests Sale Womens Further Reductions Women's Clothing Dresses Coats & Jackets Tops
Link: https://shop.bmw.co.uk/, Deceptive?: 0, Pattern String: Official BMW Shop - Genuine Parts, Accessories & Lifestyle Skip to main content Welcome to the BMW

In [42]:
#drop all the rows in without_bigcommerce_updated.csv where the "Deceptive?" column is 1

# Open the input CSV file for reading
with open('without_bigcommerce_updated.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('without_bigcommerce_without_deceptive.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Deceptive?" column is not "1"
            if row['Deceptive?'] != '1':
                # Write the row to the output CSV file
                csv_writer.writerow(row)


In [44]:
#drop all rows in bigcommerce.csv where pattern string column is empty

# Open the input CSV file for reading
with open('bigcommerce.csv', 'r') as input_file:
    csv_reader = csv.DictReader(input_file)

    # Open the output CSV file for writing
    with open('bigcommerce_without_empty_pattern.csv', 'w', newline='') as output_file:
        csv_writer = csv.DictWriter(output_file, fieldnames=csv_reader.fieldnames)
        csv_writer.writeheader()

        # Iterate over the rows of the input CSV file
        for row in csv_reader:
            # Check if the "Pattern String" column is not empty
            if row['Pattern String']:
                # Write the row to the output CSV file
                csv_writer.writerow(row)



In [48]:
import csv

def append_rows_to_combined(file_path, combined_rows):
    with open(file_path, 'a', newline='') as combined_file:
        fieldnames = ['Link', 'Pattern String', 'Deceptive?']
        combined_writer = csv.DictWriter(combined_file, fieldnames=fieldnames)

        # Append the rows to the output file
        combined_writer.writerows(combined_rows)

def main():
    combined_rows = []



    # Read rows from dark-patterns.csv (index 400 to 499)
    with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file:
        dark_pattern_reader = csv.DictReader(dark_pattern_file)
        start_index = 400
        end_index = 499
        dark_pattern_rows = list(dark_pattern_reader)[start_index:end_index + 1]

        # Append the selected rows from dark patterns to the combined rows list
        for dark_pattern_row in dark_pattern_rows:
            row = {
                'Link': dark_pattern_row['Website Page'],
                'Pattern String': dark_pattern_row['Pattern String'],
                'Deceptive?': '1'  # Mark as deceptive
            }
            combined_rows.append(row)

    # Append data from ecommerce_links_updated_without_empty_pattern_string.csv
    with open('./sample_data/ecommerce_links_updated_without_empty_pattern_string.csv', 'r') as ecommerce_links_file:
        ecommerce_links_reader = csv.DictReader(ecommerce_links_file)
        combined_rows.extend(ecommerce_links_reader)

    # Append data from bigcommerce.csv
    with open('./bigcommerce_without_empty_pattern.csv', 'r') as bigcommerce_file:
        bigcommerce_reader = csv.DictReader(bigcommerce_file)
        combined_rows.extend(bigcommerce_reader)

    # Append the combined rows to combined_datsa.csv
    append_rows_to_combined('combined_datsa_without_empty_pattern_string.csv', combined_rows)

if __name__ == "__main__":
    main()


In [49]:
# prompt: count the number of rows where Deceptive? column is 1 and when Deceptive? is 0 and print both the values

import csv

# Open the CSV file for reading
with open('combined_datsa_without_empty_pattern_string.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    # Initialize counters for deceptive and non-deceptive rows
    deceptive_count = 0
    non_deceptive_count = 0

    # Iterate over the rows in the CSV file
    for row in csv_reader:
        # Check the value of the 'Deceptive?' column
        if row['Deceptive?'] == '1':
            deceptive_count += 1
        else:
            non_deceptive_count += 1

# Print the counts
print(f"Number of deceptive rows: {deceptive_count}")
print(f"Number of non-deceptive rows: {non_deceptive_count}")


Number of deceptive rows: 283
Number of non-deceptive rows: 492


In [50]:
import csv

def append_rows_to_combined(file_path, combined_rows):
    with open(file_path, 'a', newline='') as combined_file:
        fieldnames = ['Link', 'Pattern String', 'Deceptive?']
        combined_writer = csv.DictWriter(combined_file, fieldnames=fieldnames)

        # Append the rows to the output file
        combined_writer.writerows(combined_rows)

def main():
    # Open the input CSV file
    with open('./sample_data/dark-patterns.csv', 'r') as dark_pattern_file:
        dark_pattern_reader = csv.DictReader(dark_pattern_file)

        # Set the desired index range
        start_index = 600
        end_index = 799

        # Read rows from dark-patterns.csv (index 600 to 699)
        dark_pattern_rows = list(dark_pattern_reader)[start_index:end_index + 1]

        # Create a list to store combined rows
        combined_rows = []

        # Append the selected rows from dark patterns to the combined rows list
        for dark_pattern_row in dark_pattern_rows:
            row = {
                'Link': dark_pattern_row['Website Page'],
                'Pattern String': dark_pattern_row['Pattern String'],
                'Deceptive?': '1'  # Mark as deceptive
            }
            combined_rows.append(row)

        # Append the combined rows to combined_datsa.csv
        append_rows_to_combined('combined_datsa_without_empty_pattern_string.csv', combined_rows)

if __name__ == "__main__":
    main()


In [51]:
#  drop all rows where the patterns string is empty

import csv


def drop_empty_pattern_strings(input_file, output_file):
    """
    Drops rows with empty 'Pattern String' values from the input CSV file and saves the updated data to the output file.

    Args:
        input_file: Path to the input CSV file.
        output_file: Path to the output CSV file.
    """

    with open(input_file, 'r') as file_in, open(output_file, 'w', newline='') as file_out:
        reader = csv.DictReader(file_in)
        writer = csv.DictWriter(file_out, reader.fieldnames)
        writer.writeheader()

        for row in reader:
            if row['Pattern String']:
                writer.writerow(row)


# Example usage:
input_file = 'combined_datsa_without_empty_pattern_string.csv'
output_file = 'combined_datsa_without_empty_pattern_string_updated.csv'
drop_empty_pattern_strings(input_file, output_file)



In [53]:
#  drop all the  rows with same Link column entry

import csv

def drop_duplicate_links(input_file, output_file):
    """
    Drops rows with duplicate 'Link' values from the input CSV file and saves the updated data to the output file.

    Args:
        input_file: Path to the input CSV file.
        output_file: Path to the output CSV file.
    """

    unique_links = set()
    with open(input_file, 'r') as file_in, open(output_file, 'w', newline='') as file_out:
        reader = csv.DictReader(file_in)
        writer = csv.DictWriter(file_out, reader.fieldnames)
        writer.writeheader()

        for row in reader:
            link = row['Link']
            if link not in unique_links:
                writer.writerow(row)
                unique_links.add(link)


# Example usage:
input_file = 'combined_datsa_without_empty_pattern_string_updated.csv'
output_file = 'combined_datsa_without_duplicates.csv'
drop_duplicate_links(input_file, output_file)


In [54]:
# print the number of rows with deceptive value equal to 1 and 0

import csv

with open('combined_datsa_without_duplicates.csv', 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)

    deceptive_count = 0
    non_deceptive_count = 0

    for row in csv_reader:
        if row['Deceptive?'] == '1':
            deceptive_count += 1
        else:
            non_deceptive_count += 1

print(f"Number of deceptive rows: {deceptive_count}")
print(f"Number of non-deceptive rows: {non_deceptive_count}")


Number of deceptive rows: 186
Number of non-deceptive rows: 196
